In [1]:
from collections import Counter
import pandas as pd
import stanza
import re

In [2]:
en_gt = pd.read_csv('../ACTER/en/htfl/annotations/htfl_en_terms.ann', sep='	', engine='python',header=None)
en_preds = pd.read_csv('../results/en/ann/roberta_ann_predictions_3035.csv')
preds = list(en_preds['predictions'])
gts = list(en_gt[0])

In [3]:
def count_freq(preds, gts):
    preds_len = [len(x.split(' ')) for x in preds]
    gts_len = [len(x.split(' ')) for x in gts]
    print(Counter(preds_len))
    print(Counter(gts_len))

In [4]:
preds_len = [len(x.split(' ')) for x in preds]
gts_len = [len(x.split(' ')) for x in gts]
print(Counter(preds_len))
print(Counter(gts_len))

Counter({2: 1059, 1: 718, 3: 50, 4: 2})
Counter({1: 1064, 2: 766, 3: 357, 4: 120, 5: 31, 6: 18, 7: 4, 8: 1})


In [5]:
max(gts, key=len)

'heart failure with preserved left ventricular ejection fraction'

In [6]:
# gts[gts.str.split(' ').len() == 3]

In [7]:
# P = TP/(TP+FP)
# R =  TP/(TP+FN)
def evaluation_metrics(pred, gt):
    TP = len(set(pred) & set(gt)) 
    FP = len(set(pred)-set(gt))
    FN = len(set(gt)-set(pred))
    print(TP, FP, FN)
    precision = round((TP/(TP+FP))*100, 2)
    recall = round((TP/(TP+FN))*100,2)
    f1_score = round((2 * precision * recall) / (precision + recall),2)
    return precision, recall, f1_score

precision, recall, f1_score = evaluation_metrics(preds, gts)
print(len(set(preds)))
print(len(set(gts)))
precision, recall, f1_score

636 1193 1725
1829
2361


(34.77, 26.94, 30.36)

Stanza

In [8]:
nlp = stanza.Pipeline(lang='en')# processors='tokenize,mwt,pos,lemma'

2021-04-22 15:01:20 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | combined  |
| pos       | combined  |
| lemma     | combined  |
| depparse  | combined  |
| sentiment | sstplus   |
| ner       | ontonotes |

2021-04-22 15:01:20 INFO: Use device: cpu
2021-04-22 15:01:20 INFO: Loading: tokenize
2021-04-22 15:01:20 INFO: Loading: pos
2021-04-22 15:01:20 INFO: Loading: lemma
2021-04-22 15:01:20 INFO: Loading: depparse
2021-04-22 15:01:21 INFO: Loading: sentiment
2021-04-22 15:01:21 INFO: Loading: ner
2021-04-22 15:01:22 INFO: Done loading processors!


In [9]:
def lemma(li):
    new_list = []
    for t in li:
        doc = nlp(str(t))
        doc1 = ' '.join([word.lemma for sent in doc.sentences for word in sent.words])
        doc1 = re.sub('-',' ',doc1)
        doc1 = re.sub(' +', ' ',doc1)
        new_list.append(doc1)
    new_list = [s for s in new_list if len(s) >= 2]
    return new_list

In [10]:
preds_lem = lemma(preds)
gts_lem = lemma(gts)

print('After lemmatization')
print(len(set(preds_lem)))
print(len(set((gts_lem))))
print(evaluation_metrics(list(set(preds_lem)), list(set(gts_lem))))

After lemmatization
1776
2149
608 1168 1541
(34.23, 28.29, 30.98)


In [11]:
set(preds_lem)

{'diabetic patient',
 'various transcatheter',
 'for',
 'less',
 'beat',
 'porcine',
 'related cachexia',
 'eras',
 'chm',
 'reticular',
 'tricuspid annular plane',
 'cardiac murmur',
 'lipid',
 'potassium',
 'cardiorespiratory fitness',
 'energy',
 'sphingosine 1 phosphate',
 'hemoglobin',
 'magnetic resonance',
 'cachectic',
 'rotor',
 'cardiomyopathy questionnaire',
 'self',
 'diastolic end',
 'explant',
 'tachycardia storm',
 'vasodilatory',
 'vasopressin receptor antagonist',
 'acute dyspnea',
 'transcriptional',
 'sacubiltril',
 'automatic',
 'norepinephrine',
 'cardiac tissue',
 'cardiovascular morbidity',
 'muscle cell',
 'peak vo2',
 'dynamic',
 'flow path',
 'atrial area',
 'poor myocardial',
 'transseptal',
 'transplantation',
 'disease epidemiology',
 'necrosis factor',
 'bromodomain protein',
 'flow velocity',
 'valve tethering',
 'simple ecg',
 'ventricular wall',
 'myocardial tissue',
 'bradyarrhythmias',
 'embryonic',
 'work',
 'axis function',
 'arterial pressure',
 'e

In [12]:
set(gts_lem)

{'lv relaxation',
 'repolarization',
 'adenylyl cyclase',
 'genotype',
 'psychosocial',
 'ulcer',
 'medical history',
 'afterload',
 'cell',
 'proanp',
 'nicorandil',
 'reduce ef',
 'intravenous',
 'rate ratio',
 'emergency department',
 'lean mass quantification',
 'β adrenergic receptor agonist',
 'gadolinium chelate contrast agent',
 'tricuspid annular plane',
 'cardiac murmur',
 'power spectral analysis',
 'il6',
 'energy substrate metabolism',
 'lipid',
 'clinic',
 'endothelial no synthase activation',
 'cardiorespiratory fitness',
 'potassium',
 'dyssynchronous heart failure',
 'orally',
 'sensitivity',
 'sphingosine 1 phosphate',
 'hemoglobin',
 'heart failure and reduce ejection fraction',
 'avp',
 'vascular endothelial growth factor blockade',
 '8 fluo camp',
 'explant',
 'western blot',
 'vasodilatory',
 'ss31',
 'vasopressin receptor antagonist',
 'acute dyspnea',
 'hrv',
 'lv end diastolic dimension',
 'correlate',
 'sacubiltril',
 'co morbidity',
 'partial pressure of end 

In [13]:
count_freq(set(preds_lem), set(gts_lem))

Counter({2: 1049, 1: 677, 3: 48, 4: 2})
Counter({1: 857, 2: 672, 3: 379, 4: 148, 5: 46, 6: 30, 7: 13, 8: 3, 10: 1})


In [14]:
preds

['brachial index',
 'lower limbs',
 'exercise',
 'tract infection',
 'comorbid',
 'hypertrophic response',
 'advanced heart failure',
 'energy expenditure',
 'treatment',
 'stress echocardiography',
 'with',
 'free success',
 'infusion ,',
 'tumors',
 'atherosclerosis risk',
 'medicare',
 'recipients bridged',
 'myocyte',
 'shunt',
 'bypass grafting',
 'implantation',
 'treadmill',
 'necrosis factor',
 'chemotherapy',
 'breathing',
 'cardiomyopathy questionnaire',
 'walking distance',
 'feeding',
 'inflow velocity',
 'systemic',
 'gadolinium',
 'catabolic isoenzymes',
 'transfer',
 'standard cardiac',
 'contracting ventricles',
 'vad',
 'transfusion',
 'index',
 'primary oscillations',
 'angina',
 'hf progression',
 'bottleneck stent',
 'ecg',
 'rheumatic',
 'vascular resistance',
 'patients with',
 'atrial pressure',
 'reticular',
 'technical advisors',
 'gtpase',
 'the link',
 'tnf antagonist',
 'embolectomy',
 'peak work',
 'molecule inhibitors',
 'oxidant defence',
 'cardiac hypert

In [15]:
gts

['(123)i-meta-iodobenzylguanidine',
 '(123)i-mibg',
 '1-alkyl-2-acetyl-sn-glycerol cholinephosphotransferase',
 '2-vessel cad',
 '2-way anova',
 '3-vessel cad',
 '6 min walk test',
 '6-min walk test',
 '6-minute hall walk',
 '6-minute walk distance',
 '6-minute walk test',
 '6-minute walked distance',
 '6-minute walking distance',
 '6-mwt',
 '6mwt',
 '8-fluo-camp',
 'aac',
 'abcc2',
 'abcd classification',
 'abdominal',
 'abdominal aortic constriction',
 'ablation',
 'absorptiometry',
 'acetyl-lysine reader proteins',
 'acetylcholinesterase inhibition',
 'acetylcholinesterase inhibitor',
 'acf',
 'acr',
 'acs',
 'actin cytoskeleton',
 'acute',
 'acute care surgery',
 'acute coronary syndrome',
 'acute coronary syndromes',
 'acute dyspnea',
 'acute heart failure',
 'acute hf',
 'acute ischemic right heart failure',
 'acute kidney injury',
 'acute myocardial infarction',
 'acute right heart failure',
 'acute rv failure',
 'adenine nucleotides',
 'adenosine',
 'adenoviral',
 'adenovirus',